### Testando o modo equisólido


In [ ]:
import pvlib
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
import csv
import scipy.misc
import seaborn as sns
from os import listdir
from PIL import Image
from PIL.ExifTags import TAGS
from sklearn.linear_model import LinearRegression
from pathlib import Path
from os.path import isfile,join
#from teste_calibraçao import *
from pvlib import pvsystem,location,modelchain 
from pvlib import solarposition 
import time
from scipy import stats 
from sklearn import datasets, linear_model , metrics
from sklearn import linear_model
from defisheye import Defisheye #retira a distorção associada as lentes all-sky
import warnings
import imghdr
warnings.filterwarnings("ignore")

Recortando as imagens no formato equisólido

In [ ]:
dtype ='equalarea'
format ='circular'
fov=180
pfov=120
xcenter = 768
ycenter = 768
angle = 287


In [ ]:
os.getcwd() #Saber qual diretório estou e o caminho para ele

In [ ]:
##Ler e realizar o display da imagem
from numpy import angle
figuras = os.listdir('Imagens_ajuste_r2/')[0:74]
print(figuras)
#print(imagem)
for imagem in figuras:
    cortar = f'Imagens_equisolidas_{dtype}_{format}_{pfov}_{fov}_' + imagem
    obj =Defisheye('Imagens_ajuste_r2/' + imagem,dtype=dtype,format=format,fov=fov,pfov=pfov,xcenter=xcenter,ycenter=ycenter,angle=angle)
    obj.convert(cortar)
    print(obj)


Marcando a posição do sol na imagem e escodendo a posição do sol na imagem

In [ ]:
cards=[]
img = os.listdir('Imagens_equisolidas/')[0:74]
print(img)
for nome_imagem in img:
    im =cv2.imread('Imagens_equisolidas/' + nome_imagem)
    im =cv2.flip(im,1)  
    print(im)

    red=im[:,:,2]
    limite = 254
    nred = np.where(red > limite , red, 0)
    #print(nred)
    z = np.where(nred >0 ) #Pontos onde os valores de nred forem maiores que 0
    #print('Quantidade de pontos acima do limite', limite, '=', z[0].size)
    #identificando posições em que a componente vermelha seja superior ao limite
    z = np.where(red>limite)
    #print(z)
    length = np.shape(z)[1]
    #print('O tamanho é igual a:' , length)
    xc = int(z[0].mean())
    yc = int(z[1].mean())
    print('posição do centroide = (',xc,yc,') , quantidade de pontos acima do limite:', length)
    
    dist = (((z[0]-int(z[0].mean()))**2)+((z[1]-int(z[1].mean()))**2))**0.5
    
    temp = np.where(dist<=np.median(dist))
    np.percentile(dist,75)

    percentil = 90
    temp= np.where(dist <= np.percentile(dist,percentil))
    #print(r)
    j=0
    ndist=np.zeros(np.shape(temp)[1])
    z2 = [np.zeros(np.shape(temp)[1], dtype=int), np.zeros(np.shape(temp)[1], dtype=int)]

    Limite_d = np.percentile(dist,percentil)
    #print (Limite_d)
    for i in range(0,int(length)):
        if dist[i] <= Limite_d:
            ndist[j]=dist[i]
            z2[0][j]=int(z[0][i])
            z2[1][j]=int(z[1][i])
            j=j+1

    ## Centroide atualizado
    xc_z2 = int(z2[0].mean())
    yc_z2 = int(z2[1].mean())
    print('Centroide atualizado= (',xc_z2,yc_z2,')')

    nnread = im
    print(xc_z2,yc_z2)
    #for i in range(0,len(z2[0])):
    #    nnread[z2[0][i],z2[1][i],[2,1,0]]=0
    plt.figure(figsize=(12,12))
    plt.xlabel('Posição em Y')
    plt.ylabel('Posição em X')
    plt.title('Calibrada ' + nome_imagem)
    plt.scatter(x=yc_z2, y=xc_z2, c='r', s=100)
    plt.imshow(im[:,:,[2,1,0]])
    plt.savefig('Calibra_recortada/' + 'calibrada_defisheye' + nome_imagem)
    plt.show()


    window_name = 'Imagem'
    center_coodinates =(yc_z2,xc_z2) #teve que inverter os pontos
    radius = 140
    color = (0,0,0)
    thickness= -1
    im = cv2.circle(im,center_coodinates,radius,color,thickness)
    plt.figure(figsize=(12,12))
    plt.xlabel('Posição em Y')
    plt.ylabel('Posição em X')
    plt.imshow(im[:,:,[2,1,0]])
    #cv2.imshow('Imagem espelhada', im)
    plt.savefig('sol_encoberto/' + 'area_140' +nome_imagem)
    plt.show()


    ### Definir a área circular do R fixo para plotar o circulo
    #window_name = 'Imagem'
    #center_coodinates =(yc_z2,xc_z2) #teve que inverter os pontos
    #radius = 140
    #color = (0,0,0)
    #thickness= -1
    #im = cv2.circle(im,center_coodinates,radius,color,thickness)
    #plt.figure(figsize=(12,12))
    #plt.xlabel('Posição em Y')
    #plt.ylabel('Posição em X')
    #plt.imshow(im[:,:,[2,1,0]])
    ##cv2.imshow('Imagem espelhada', im)
    #plt.savefig('Calibracao_Algoritmo/' + 'area_140' +figura)
    #plt.show()
    
    card = {}
    card['num linhas'] = im.shape[0]
    card['num colunas'] = im.shape[1]
    card['num canais'] = im.shape[2]
    card["Endereço da imagem"] = "Calibração Recortada" + nome_imagem
    card["Posição em X"] = yc_z2
    card["Posição em Y"] = xc_z2
    card["Posição em X²"] = (yc_z2)**2
    card['Posição em Y²'] = (xc_z2)**2
    card['Posição em X afetada']= yc
    card['Posição em Y afetada']= xc
    card['Distancia máxima'] = ndist.max()
    card['Distancia média'] = dist.mean()
    card['Distância mínima'] = ndist.min()
    card['Valor de R'] = temp
    cards.append(card)


In [ ]:
df_info_defisheye= pd.DataFrame(cards)
df_info_defisheye
df_info_defisheye.to_csv(r'informações_defisheye.csv', index  = False)

In [ ]:
df_info_defisheye

### Organizando os blocos de tempo para todo o período

Calculando o zênite e o azimute da imagem recortada

In [ ]:
import pytz
pytz.all_timezones

In [ ]:
time_1= pd.date_range(start='2020-03-17 11:30:00', end='2020-03-17 12:30:00', periods=3, tz='America/Fortaleza')
time_2= pd.date_range(start='2020-03-18 12:00:00', end='2020-03-18 12:30:00', periods=2, tz='America/Fortaleza')
time_3= pd.date_range(start='2020-03-21 11:30:00', end='2020-03-21 12:30:00', periods=3, tz='America/Fortaleza')
time_4= pd.date_range(start='2020-03-26 12:00:00', end='2020-03-26 13:30:00', periods=4, tz='America/Fortaleza')
time_5= pd.date_range(start='2020-07-03 12:00:00', end='2020-07-03 15:00:00', periods=7, tz='America/Fortaleza')
time_6= pd.date_range(start='2020-07-07 12:00:00', end='2020-07-07 16:00:00', periods=9, tz='America/Fortaleza')
time_7= pd.date_range(start='2020-07-08 11:30:00', end='2020-07-08 16:00:00', periods=10, tz='America/Fortaleza')
time_8= pd.date_range(start='2020-07-13 12:00:00', end='2020-07-13 16:00:00', periods=9, tz='America/Fortaleza')
time_9= pd.date_range(start='2020-07-24 11:30:00', end='2020-07-24 15:30:00', periods=9, tz='America/Fortaleza')
time_10= pd.date_range(start='2020-07-28 12:30:00', end='2020-07-28 16:00:00', periods=8, tz='America/Fortaleza')
time_11= pd.date_range(start='2020-07-30 11:30:00', end='2020-07-30 16:00:00', periods=10, tz='America/Fortaleza')

In [ ]:
list_times = list([time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,time_10,time_11])
len(list_times) #descobrir o tamanho da lista

In [ ]:
list_times

In [ ]:
df_za = pd.DataFrame()
for time in list_times:
    data_az= solarposition.get_solarposition(time, -5.79, -35.21, altitude=0, pressure=101325)
    data_az = data_az.loc[data_az['apparent_elevation'] > 0 ,:]
    ax  = plt.subplot(1,1,1, projection ='polar')

    # draw the analemma loops
    points = ax.scatter(np.radians(data_az.azimuth), data_az.apparent_zenith,
                    s=2, label=None, c=data_az.index.dayofyear)
    ax.figure.colorbar(points)

    # draw hour labels
    for hour in np.unique(data_az.index.hour):
        # choose label position by the smallest radius for each hour 
        subset = data_az.loc[data_az.index.hour == hour, :]
        r = subset.apparent_zenith
        pos = data_az.loc[r.idxmin(), :]
        ax.text(np.radians(pos['azimuth']), pos['apparent_zenith'], str(hour))


    ax.figure.legend(loc='upper left')
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_rmax(90)

    plt.show()

    data_az['time'] = time
    
    df_za = pd.concat([df_za,data_az], axis=0)
    
df_za.to_csv('azimuth_and_zenith_calibration_new.csv', index = False) # concatenaçao a cada data_az criado (pd.concat)
print('A localização do sol na imagem é: ' , df_za)

In [ ]:
df_za.shape #saber o tamanho da variável

In [ ]:
df_za

In [ ]:
df_za['azimuth_convert']=df_za['azimuth'] - 360
df_za


In [ ]:
df_info_defisheye = pd.DataFrame(cards)
df_info_defisheye['Posição em X com base nas linhas'] = (df_info_defisheye['Posição em X'] - (df_info_defisheye['num linhas']/2))
df_info_defisheye['Posição em Y com base nas colunas'] = (df_info_defisheye['Posição em Y'] - (df_info_defisheye['num colunas']/2))
df_info_defisheye

Organizando as informações do Data Frame

In [ ]:
df_info_defisheye['hipotenusa'] = (df_info_defisheye['Posição em X com base nas linhas'])**2 + (df_info_defisheye['Posição em Y com base nas colunas'])**2
df_info_defisheye['raiz_hipotenusa'] = df_info_defisheye['hipotenusa'] ** 0.5
df_info_defisheye['cos_phi'] = df_info_defisheye['Posição em X com base nas linhas'] / df_info_defisheye['raiz_hipotenusa']
df_info_defisheye['cos_phi']
df_info_defisheye['sen_phi'] = df_info_defisheye['Posição em Y com base nas colunas']/ df_info_defisheye['raiz_hipotenusa']
df_info_defisheye['sen_phi']
df_info_defisheye['tg_phi'] = df_info_defisheye['sen_phi'] /df_info_defisheye['cos_phi']
df_info_defisheye['tg_phi']
arc_tg = np.array(df_info_defisheye['tg_phi'])
arc_tg
df_info_defisheye['arctg_phi'] = np.arctan(arc_tg) 


In [ ]:
df_info_defisheye

In [ ]:
df_info_defisheye.shape

In [ ]:
df_info_defisheye.to_csv(r'informações_defisheye.csv', index  = False)

In [ ]:
df_za.to_csv(r'Zenite_Azimute_im_recortada.csv', index = False)

Unindo as informações do sol na imagem com a posição com base no zênte e no azimute

In [ ]:
df_defisieye_zen_azim = pd.concat([df_info_defisheye,df_za], axis = 0)

In [ ]:
df_defisieye_zen_azim

In [ ]:
df_defisieye_zen_azim.to_csv(r'Defisheye_zen_azim.csv', index = False)

### Regressão Linear para avaliar o ajuste

Criando a coluna da posição do sol na imagem e teta/2

In [ ]:
zenite = np.array(np.sin((df_za['zenith'])*np.pi/360))
sol_defish = np.array(df_info_defisheye['raiz_hipotenusa'])
print(zenite)
print(sol_defish)

In [ ]:
sol_defish.shape

In [ ]:
cards = []
regressao = linear_model.LinearRegression()
zenite = zenite.reshape(-1,1)
print(zenite)
regressao.fit(zenite,sol_defish)

correlacao_zsol = regressao.score(zenite,sol_defish)
print('o valor da correlação é:', correlacao_zsol)
k1 = regressao.coef_
print('o valor do coeficiente, ou seja, k1 é:', k1)
k2 = regressao.intercept_
print('O valor do intercepto, ou seja, k2 é:', k2)

card ={}
card['Correlação'] = correlacao_zsol
card['Coeficiente = K1'] = k1
card['Intercepto = K2'] = k2
cards.append(card)

df_regr_ajustado = pd.DataFrame(cards)
df_regr_ajustado

df_regr_ajustado.to_csv(r'score_coeficiente_intercepto_defisheye.csv', index = False)


In [ ]:
sns.regplot(zenite,sol_defish, ci=None)

plt.xlabel('Sen (θ/2)')
plt.ylabel('Posição do sol (rad)')
plt.savefig('Regressões/' + "Reg_defisheye_zen_azi" )

plt.show()